In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import numpy as np

In [2]:
#  Import and read the flight data for 2018.
flight_18_df = pd.read_csv("Data_files/2018.csv")
flight_18_df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Check data types. 
flight_18_df.info()

In [3]:
# Convert flight date to datetime. 
flight_18_df["FL_DATE"] = pd.to_datetime(flight_18_df["FL_DATE"])

In [4]:
# Separate the datetime column into columns for year, month, day, and weekday. 
flight_18_df["YEAR"]=flight_18_df["FL_DATE"].dt.year
flight_18_df["MONTH"]=flight_18_df["FL_DATE"].dt.month
flight_18_df["DAY"]=flight_18_df["FL_DATE"].dt.day
# flight_18_df["WEEKDAY"]=flight_18_df["FL_DATE"].dt.dayofweek
flight_18_df["WEEKDAY"]=flight_18_df['FL_DATE'].dt.strftime('%A')

In [ ]:
list(flight_18_df.columns)

In [5]:
# Drop columns that we won't need for our models. 
prio_flight_18_df = flight_18_df.drop(["FL_DATE", "OP_CARRIER_FL_NUM", "DIVERTED", "CANCELLATION_CODE", "WEATHER_DELAY", "LATE_AIRCRAFT_DELAY", "CARRIER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "Unnamed: 27", "CANCELLED"],axis=1)
prio_flight_18_df.head()

,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,...,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,YEAR,MONTH,DAY,WEEKDAY
0,UA,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,...,1722.0,-23.0,268.0,250.0,225.0,1605.0,2018,1,1,Monday
1,UA,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,...,1230.0,-24.0,99.0,83.0,65.0,414.0,2018,1,1,Monday
2,UA,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,...,1636.0,-13.0,134.0,126.0,106.0,846.0,2018,1,1,Monday
3,UA,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,6.0,...,1754.0,-2.0,190.0,182.0,157.0,1120.0,2018,1,1,Monday
4,UA,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,10.0,...,936.0,14.0,112.0,106.0,83.0,723.0,2018,1,1,Monday


In [6]:
# Fill all null values with zeros. 
prio_flight_18_df = prio_flight_18_df.fillna(0)

In [7]:
# Categorize the scheduled DEPARTURE time to four key times of day. 
prio_flight_18_df.loc[(prio_flight_18_df["CRS_DEP_TIME"] >= 0) & (prio_flight_18_df["CRS_DEP_TIME"] < 600), "sched_dep_time"] = "Night"
prio_flight_18_df.loc[(prio_flight_18_df["CRS_DEP_TIME"] >= 600) & (prio_flight_18_df["CRS_DEP_TIME"] < 1200), "sched_dep_time"] = "Morning"
prio_flight_18_df.loc[(prio_flight_18_df["CRS_DEP_TIME"] >= 1200) & (prio_flight_18_df["CRS_DEP_TIME"] < 1800), "sched_dep_time"] = "Afternoon"
prio_flight_18_df.loc[(prio_flight_18_df["CRS_DEP_TIME"] >= 1800) & (prio_flight_18_df["CRS_DEP_TIME"] < 2400), "sched_dep_time"] = "Evening"

In [8]:
# Categorize the scheduled ARRIVAL time to four key times of day. 
prio_flight_18_df.loc[(prio_flight_18_df["CRS_ARR_TIME"] >= 0) & (prio_flight_18_df["CRS_ARR_TIME"] < 600), "sched_arr_time"] = "Night"
prio_flight_18_df.loc[(prio_flight_18_df["CRS_ARR_TIME"] >= 600) & (prio_flight_18_df["CRS_ARR_TIME"] < 1200), "sched_arr_time"] = "Morning"
prio_flight_18_df.loc[(prio_flight_18_df["CRS_ARR_TIME"] >= 1200) & (prio_flight_18_df["CRS_ARR_TIME"] < 1800), "sched_arr_time"] = "Afternoon"
prio_flight_18_df.loc[(prio_flight_18_df["CRS_ARR_TIME"] >= 1800) & (prio_flight_18_df["CRS_ARR_TIME"] < 2400), "sched_arr_time"] = "Evening"

In [ ]:
prio_flight_18_df.head(20)

In [9]:
# Categorize the months column by season. 
prio_flight_18_df.loc[(prio_flight_18_df["MONTH"] == 12) | (prio_flight_18_df["MONTH"] == 1) | (prio_flight_18_df["MONTH"] == 2), "season"] = "Winter"
prio_flight_18_df.loc[(prio_flight_18_df["MONTH"] > 2) & (prio_flight_18_df["MONTH"] <= 5), "season"] = "Spring"
prio_flight_18_df.loc[(prio_flight_18_df["MONTH"] > 5) & (prio_flight_18_df["MONTH"] <= 8), "season"] = "Summer"
prio_flight_18_df.loc[(prio_flight_18_df["MONTH"] > 8) & (prio_flight_18_df["MONTH"] <= 11), "season"] = "Autumn"

In [10]:
# Categorize each flight as delayed or not, based on if the flight arrived at its destination late. 
prio_flight_18_df['DELAY'] = np.where((prio_flight_18_df.ARR_DELAY > 0), 1, 0)
prio_flight_18_df.head()

,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,...,AIR_TIME,DISTANCE,YEAR,MONTH,DAY,WEEKDAY,sched_dep_time,sched_arr_time,season,DELAY
0,UA,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,...,225.0,1605.0,2018,1,1,Monday,Afternoon,Afternoon,Winter,0
1,UA,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,...,65.0,414.0,2018,1,1,Monday,Morning,Afternoon,Winter,0
2,UA,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,...,106.0,846.0,2018,1,1,Monday,Afternoon,Afternoon,Winter,0
3,UA,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,6.0,...,157.0,1120.0,2018,1,1,Monday,Afternoon,Afternoon,Winter,0
4,UA,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,10.0,...,83.0,723.0,2018,1,1,Monday,Morning,Morning,Winter,1


In [ ]:
prio_flight_18_df.info()

In [11]:
# Drop remaining time- and delay-related columns and columns that might lead to multi-collinearity or model confusion:
prio_flight_18_df = prio_flight_18_df.drop(["DEP_TIME", "DEP_DELAY", "TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "TAXI_IN", "ARR_TIME", "ARR_DELAY", "ACTUAL_ELAPSED_TIME", "AIR_TIME", "MONTH", "DAY", "CRS_ARR_TIME", "CRS_DEP_TIME"], axis=1)

In [12]:
# Check remaining columns: 
list(prio_flight_18_df.columns)

['OP_CARRIER',
 'ORIGIN',
 'DEST',
 'CRS_ELAPSED_TIME',
 'DISTANCE',
 'YEAR',
 'WEEKDAY',
 'sched_dep_time',
 'sched_arr_time',
 'season',
 'DELAY']

In [13]:
# Determine the top 5 airports so we can narrow the scope of the analysis. 
prio_flight_18_df["ORIGIN"].value_counts()

ATL    390046
ORD    332953
DFW    279298
DEN    235989
CLT    233317
        ...  
AKN        63
CYS        58
IFP        45
ART        25
YNG         2
Name: ORIGIN, Length: 358, dtype: int64

In [14]:
# Determine the top 5 airports so we can narrow the scope of the analysis. 
prio_flight_18_df["DEST"].value_counts()

ATL    390079
ORD    332942
DFW    279272
DEN    236020
CLT    233309
        ...  
AKN        63
CYS        58
IFP        45
ART        26
YNG         2
Name: DEST, Length: 358, dtype: int64

In [15]:
# Determine the top 5 airlines so we can narrow the scope of the analysis. 
prio_flight_18_df["OP_CARRIER"].value_counts()

WN    1352552
DL     949283
AA     916818
OO     774137
UA     621565
YX     316090
B6     305010
MQ     296001
OH     278457
9E     245917
AS     245761
YV     215138
EV     202890
NK     176178
F9     120035
G4      96221
HA      83723
VX      17670
Name: OP_CARRIER, dtype: int64

In [16]:
# Create lists to represent the top 5 airports and airlines. 
top_airports = ["ATL","ORD","DFW","CLT","DEN"]
top_airlines = ["WN", "DL", "AA", "OO", "UA"]

In [17]:
# Limit the data frame to include only the top 5 airports and airlines. 
top_prio_flight_18_df = prio_flight_18_df.loc[prio_flight_18_df['ORIGIN'].isin(top_airports)]
top_prio_flight_18_df = top_prio_flight_18_df.loc[prio_flight_18_df['DEST'].isin(top_airports)]
top_prio_flight_18_df = top_prio_flight_18_df.loc[prio_flight_18_df['OP_CARRIER'].isin(top_airlines)]
top_prio_flight_18_df = top_prio_flight_18_df.reset_index(drop=True)

In [18]:
# Review the columns to determine which need to be scaled and which need one hot encoding. 
top_prio_flight_18_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100170 entries, 0 to 100169
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   OP_CARRIER        100170 non-null  object 
 1   ORIGIN            100170 non-null  object 
 2   DEST              100170 non-null  object 
 3   CRS_ELAPSED_TIME  100170 non-null  float64
 4   DISTANCE          100170 non-null  float64
 5   YEAR              100170 non-null  int64  
 6   WEEKDAY           100170 non-null  object 
 7   sched_dep_time    100170 non-null  object 
 8   sched_arr_time    100170 non-null  object 
 9   season            100170 non-null  object 
 10  DELAY             100170 non-null  int32  
dtypes: float64(2), int32(1), int64(1), object(7)
memory usage: 8.0+ MB


In [19]:
list(prio_flight_18_df.columns)

['OP_CARRIER',
 'ORIGIN',
 'DEST',
 'CRS_ELAPSED_TIME',
 'DISTANCE',
 'YEAR',
 'WEEKDAY',
 'sched_dep_time',
 'sched_arr_time',
 'season',
 'DELAY']

In [20]:
# Scale the numeric columns: 
flight_data_scaled = StandardScaler().fit_transform(top_prio_flight_18_df[["CRS_ELAPSED_TIME", "DISTANCE"]])

In [21]:
# Create a data frame. 
top_prio_flight_scaled = pd.DataFrame(
    flight_data_scaled,
    columns=["CRS_ELAPSED_TIME", "DISTANCE"]
)

In [22]:
# Check the data frame. 
top_prio_flight_scaled

,CRS_ELAPSED_TIME,DISTANCE
0,0.191684,0.454929
1,-0.567662,-0.589053
2,0.647291,0.454929
3,0.738413,0.454929
4,-0.324671,-0.437333
...,...,...
100165,0.130936,0.140650
100166,-0.689157,-0.589053
100167,-0.172802,0.628323
100168,-2.055979,-1.936476


In [24]:
list(prio_flight_18_df.columns)

['OP_CARRIER',
 'ORIGIN',
 'DEST',
 'CRS_ELAPSED_TIME',
 'DISTANCE',
 'YEAR',
 'WEEKDAY',
 'sched_dep_time',
 'sched_arr_time',
 'season',
 'DELAY']

In [25]:
# Get dummies for the categorical columns: 
flight_dummies = pd.get_dummies(top_prio_flight_18_df[["OP_CARRIER", "ORIGIN", "DEST", "WEEKDAY", "sched_dep_time", "sched_arr_time", "season"]])

In [26]:
# Check the data frame. 
flight_dummies

,OP_CARRIER_AA,OP_CARRIER_DL,OP_CARRIER_OO,OP_CARRIER_UA,OP_CARRIER_WN,ORIGIN_ATL,ORIGIN_CLT,ORIGIN_DEN,ORIGIN_DFW,ORIGIN_ORD,...,sched_dep_time_Morning,sched_dep_time_Night,sched_arr_time_Afternoon,sched_arr_time_Evening,sched_arr_time_Morning,sched_arr_time_Night,season_Autumn,season_Spring,season_Summer,season_Winter
0,0,0,0,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
2,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
4,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100165,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
100166,1,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
100167,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
100168,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [27]:
# Confirm dummies. 
list(flight_dummies.columns)

['OP_CARRIER_AA',
 'OP_CARRIER_DL',
 'OP_CARRIER_OO',
 'OP_CARRIER_UA',
 'OP_CARRIER_WN',
 'ORIGIN_ATL',
 'ORIGIN_CLT',
 'ORIGIN_DEN',
 'ORIGIN_DFW',
 'ORIGIN_ORD',
 'DEST_ATL',
 'DEST_CLT',
 'DEST_DEN',
 'DEST_DFW',
 'DEST_ORD',
 'WEEKDAY_Friday',
 'WEEKDAY_Monday',
 'WEEKDAY_Saturday',
 'WEEKDAY_Sunday',
 'WEEKDAY_Thursday',
 'WEEKDAY_Tuesday',
 'WEEKDAY_Wednesday',
 'sched_dep_time_Afternoon',
 'sched_dep_time_Evening',
 'sched_dep_time_Morning',
 'sched_dep_time_Night',
 'sched_arr_time_Afternoon',
 'sched_arr_time_Evening',
 'sched_arr_time_Morning',
 'sched_arr_time_Night',
 'season_Autumn',
 'season_Spring',
 'season_Summer',
 'season_Winter']

In [28]:
# Isolate variables that we're trying to predict. 
variables_predicting_df = top_prio_flight_18_df[['DELAY']]

In [29]:
# Check the data frame. 
variables_predicting_df

,DELAY
0,0
1,0
2,0
3,0
4,0
...,...
100165,0
100166,1
100167,0
100168,0


In [30]:
# Concatenate the three data frames. 
cleaned_flights_df = pd.concat([top_prio_flight_scaled, flight_dummies, variables_predicting_df], axis=1)

In [31]:
cleaned_flights_df

,CRS_ELAPSED_TIME,DISTANCE,OP_CARRIER_AA,OP_CARRIER_DL,OP_CARRIER_OO,OP_CARRIER_UA,OP_CARRIER_WN,ORIGIN_ATL,ORIGIN_CLT,ORIGIN_DEN,...,sched_dep_time_Night,sched_arr_time_Afternoon,sched_arr_time_Evening,sched_arr_time_Morning,sched_arr_time_Night,season_Autumn,season_Spring,season_Summer,season_Winter,DELAY
0,0.191684,0.454929,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
1,-0.567662,-0.589053,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0.647291,0.454929,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,0.738413,0.454929,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,-0.324671,-0.437333,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100165,0.130936,0.140650,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
100166,-0.689157,-0.589053,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
100167,-0.172802,0.628323,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
100168,-2.055979,-1.936476,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0


In [32]:
# Finally, need to drop the columns that might interfere with delay predictions: 
# Let's check our columns again: 
list(cleaned_flights_df.columns)

['CRS_ELAPSED_TIME',
 'DISTANCE',
 'OP_CARRIER_AA',
 'OP_CARRIER_DL',
 'OP_CARRIER_OO',
 'OP_CARRIER_UA',
 'OP_CARRIER_WN',
 'ORIGIN_ATL',
 'ORIGIN_CLT',
 'ORIGIN_DEN',
 'ORIGIN_DFW',
 'ORIGIN_ORD',
 'DEST_ATL',
 'DEST_CLT',
 'DEST_DEN',
 'DEST_DFW',
 'DEST_ORD',
 'WEEKDAY_Friday',
 'WEEKDAY_Monday',
 'WEEKDAY_Saturday',
 'WEEKDAY_Sunday',
 'WEEKDAY_Thursday',
 'WEEKDAY_Tuesday',
 'WEEKDAY_Wednesday',
 'sched_dep_time_Afternoon',
 'sched_dep_time_Evening',
 'sched_dep_time_Morning',
 'sched_dep_time_Night',
 'sched_arr_time_Afternoon',
 'sched_arr_time_Evening',
 'sched_arr_time_Morning',
 'sched_arr_time_Night',
 'season_Autumn',
 'season_Spring',
 'season_Summer',
 'season_Winter',
 'DELAY']

In [ ]:
# Let's remove these columns (which might impact delay predictions):
# delays_weather_no, delays_weather_yes, delays_late_aircraft_no, delays_late_aircraft_yes, CANCELLED, DIVERTED, DEP_DELAY, ARR_DELAY
# cleaned_flights_df = cleaned_flights_df.drop(["delays_weather_no", "delays_weather_yes", "delays_late_aircraft_no", "delays_late_aircraft_yes", "CANCELLED", "DIVERTED", "DEP_DELAY", "ARR_DELAY"],axis=1)

In [33]:
cleaned_flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100170 entries, 0 to 100169
Data columns (total 37 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   CRS_ELAPSED_TIME          100170 non-null  float64
 1   DISTANCE                  100170 non-null  float64
 2   OP_CARRIER_AA             100170 non-null  uint8  
 3   OP_CARRIER_DL             100170 non-null  uint8  
 4   OP_CARRIER_OO             100170 non-null  uint8  
 5   OP_CARRIER_UA             100170 non-null  uint8  
 6   OP_CARRIER_WN             100170 non-null  uint8  
 7   ORIGIN_ATL                100170 non-null  uint8  
 8   ORIGIN_CLT                100170 non-null  uint8  
 9   ORIGIN_DEN                100170 non-null  uint8  
 10  ORIGIN_DFW                100170 non-null  uint8  
 11  ORIGIN_ORD                100170 non-null  uint8  
 12  DEST_ATL                  100170 non-null  uint8  
 13  DEST_CLT                  100170 non-null  u

In [34]:
# Save the cleaned data to a CSV. 
cleaned_flights_df.to_csv("Data_files/REV_2018_cleaned_delays.csv", index=False)